In [2]:
from maverick import Card, Deck, Holding, Hand

deck = Deck.build().shuffle()
holding = Holding(cards=deck.deal(2))
hand = Hand(private_cards=holding.cards, community_cards=deck.deal(3))
hand, hand.score()

(7♥ 2♦ 4♠ J♠ A♣, (<HandType.HIGH_CARD: 1>, 14.11742))

In [3]:
deck = Deck.build().shuffle()
holding = Holding(cards=deck.deal(2))
hand = Hand(private_cards=holding.cards, community_cards=deck.deal(3))
hand, hand.score()

(Q♥ 9♦ K♣ 3♦ J♥, (<HandType.HIGH_CARD: 1>, 13.131929999999999))

In [4]:
deck = Deck.standard_deck(shuffle=True)
all_possible_holdings = list(Holding.all_possible_holdings(deck.cards, n=2))
all_possible_holdings[0]

8♦ A♦

In [4]:
deck = Deck.standard_deck(shuffle=True)
all_possible_hands = list(Hand.all_possible_hands(deck.cards))
all_possible_hands[0]

8♦ K♦ 2♠ 3♦ K♣

In [5]:
[hand.score() for hand in all_possible_hands[:5]]

[(<HandType.PAIR: 2>, 28.083199999999998),
 (<HandType.HIGH_CARD: 1>, 13.086319999999999),
 (<HandType.HIGH_CARD: 1>, 13.08532),
 (<HandType.HIGH_CARD: 1>, 13.118319999999997),
 (<HandType.HIGH_CARD: 1>, 13.098319999999998)]

In [ ]:
from maverick.utils import estimate_holding_strength

print(all_possible_holdings[0])
prob = estimate_holding_strength(all_possible_holdings[0].cards, n_simulations=1000, n_players=8)
print(f"Estimated holding strength: {prob:.2%}\n")

# Assemble a holding with two aces directly
pair_of_aces = Holding(cards=[
    Card(suit='S', rank=14),  # Ace of Spades
    Card(suit='H', rank=14)   # Ace of Hearts
])
print(pair_of_aces.cards)
prob = estimate_holding_strength(pair_of_aces.cards, n_simulations=1000, n_players=8)
print(f"Estimated holding strength: {prob:.2%}\n")

cards=[Card(suit=<Suit.CLUBS: 'C'>, rank=<Rank.FOUR: 4>), Card(suit=<Suit.SPADES: 'S'>, rank=<Rank.SEVEN: 7>)]
Estimated holding strength: 7.40%

[Card(suit=<Suit.SPADES: 'S'>, rank=<Rank.ACE: 14>), Card(suit=<Suit.HEARTS: 'H'>, rank=<Rank.ACE: 14>)]
Estimated holding strength: 73.10%



In [7]:
pair_of_aces.estimate_strength(n_simulations=1000, n_players=8)

0.73